In [85]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
!pip install gensim
from gensim.models import Word2Vec
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity  # Import cosine_similarity

# Load your existing dataset
file_path = "/content/Final_dataset.csv"  # Replace with your file path
data = pd.read_csv(file_path,encoding='latin1')


# Define possible activities and their categories
activities = {
    'amenities': ['pool', 'spa', 'gym', 'tennis_court', 'business_center'],
    'dining': ['main_restaurant', 'cafe', 'bar', 'room_service', 'buffet'],
    'activities': ['city_tour', 'beach_activity', 'cooking_class', 'yoga', 'golf']
}

# Add new columns to your dataset
random.seed(42)  # Set a seed for reproducibility
activity_data = []

for index, row in data.iterrows():
    category = random.choice(list(activities.keys()))
    activity = random.choice(activities[category])
    rating = random.randint(1, 5)
    time_spent = random.randint(30, 180)  # Minutes

    # Append new columns
    activity_data.append({
        'User_id': row['User_id'],
        'Date and Time': row['Date and Time'],
        'category': category,
        'activity': activity,
        'rating': rating,
        'time_spent': time_spent,
        'time_of_day': random.choice(['morning', 'afternoon', 'evening']),
        'weekend': pd.to_datetime(row['Date and Time']).weekday() >= 5
    })

# Convert the list to a DataFrame and merge it with the original data
activity_df = pd.DataFrame(activity_data)
final_data = pd.merge(data, activity_df, on=['User_id', 'Date and Time'])

<ipython-input-85-d47e60df6569>:42: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  'weekend': pd.to_datetime(row['Date and Time']).weekday() >= 5


In [86]:
final_data.head()

,User_id,Customer Name,Customer Mail ID,Unnamed: 3,Date and Time,Review,Server Name,Server EMPID,Customer Age,Dining Preference,Room Preference,Sports Activity,Wellness,Pricing Pattern,category,activity,rating,time_spent,time_of_day,weekend
0,1,James Craig,fmills@yahoo.com,NaN,15-03-2024 18:31,nice hotel expensive parking got good deal sta...,Christopher,EMP5675,53,Yes,Standard,No,Gym,High,activities,city_tour,1,100,morning,False
1,2,Michelle Rhodes,karenyates@robinson.info,NaN,31-03-2024 06:27,ok nothing special charge diamond member hilto...,Sharon,EMP8272,74,No,Standard,Yes,Spa,High,amenities,spa,1,169,morning,True
2,3,Yvonne Alvarez,reedcharles@davis.info,NaN,29-04-2024 06:37,nice rooms not 4* experience hotel monaco seat...,Jennifer,EMP7042,60,No,Deluxe,Yes,Pool,Medium,activities,yoga,1,37,morning,False
3,4,Jamie Pham,zbowers@aguirre.com,NaN,17-07-2024 03:25,unique \tgreat stay \twonderful time hotel mon...,William,EMP2057,41,Yes,Suite,No,Spa,Medium,amenities,spa,5,36,evening,False
4,5,Vickie George,ssmith@yahoo.com,NaN,08-02-2024 13:19,great stay great stay \twent seahawk game awes...,Jason,EMP8088,66,No,Suite,No,Pool,Low,amenities,business_center,4,86,afternoon,False


In [87]:
final_data.drop(columns=['Unnamed: 3'], inplace=True)

In [88]:
 final_data

,User_id,Customer Name,Customer Mail ID,Date and Time,Review,Server Name,Server EMPID,Customer Age,Dining Preference,Room Preference,Sports Activity,Wellness,Pricing Pattern,category,activity,rating,time_spent,time_of_day,weekend
0,1,James Craig,fmills@yahoo.com,15-03-2024 18:31,nice hotel expensive parking got good deal sta...,Christopher,EMP5675,53,Yes,Standard,No,Gym,High,activities,city_tour,1,100,morning,False
1,2,Michelle Rhodes,karenyates@robinson.info,31-03-2024 06:27,ok nothing special charge diamond member hilto...,Sharon,EMP8272,74,No,Standard,Yes,Spa,High,amenities,spa,1,169,morning,True
2,3,Yvonne Alvarez,reedcharles@davis.info,29-04-2024 06:37,nice rooms not 4* experience hotel monaco seat...,Jennifer,EMP7042,60,No,Deluxe,Yes,Pool,Medium,activities,yoga,1,37,morning,False
3,4,Jamie Pham,zbowers@aguirre.com,17-07-2024 03:25,unique \tgreat stay \twonderful time hotel mon...,William,EMP2057,41,Yes,Suite,No,Spa,Medium,amenities,spa,5,36,evening,False
4,5,Vickie George,ssmith@yahoo.com,08-02-2024 13:19,great stay great stay \twent seahawk game awes...,Jason,EMP8088,66,No,Suite,No,Pool,Low,amenities,business_center,4,86,afternoon,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,Todd York,sanchezkaitlyn@yahoo.com,01-10-2024 07:16,minor grievances dream visit paris \thoneymoon...,Francisco,EMP5606,71,No,Standard,Yes,Gym,High,dining,main_restaurant,2,162,evening,False
9996,9997,Vernon King,crystalthompson@chandler.com,07-01-2024 02:56,favorite quartier hotel stayed duquesne eiffel...,Debra,EMP4923,33,No,Standard,No,Spa,High,dining,main_restaurant,3,68,morning,False
9997,9998,Stephen Dixon,markclay@yahoo.com,30-11-2024 15:14,wonderful view cute room stayed room 55 magnif...,Robert,EMP9674,29,Yes,Deluxe,Yes,Gym,Low,dining,room_service,4,111,morning,True
9998,9999,Brian Bishop,smithisaac@gmail.com,08-12-2024 05:31,gem \tweeks looking online hotels paris say di...,Danielle,EMP2106,78,Yes,Standard,Yes,Spa,High,amenities,tennis_court,3,138,afternoon,False


In [89]:
final_data_kept = final_data[['User_id','Customer Age', 'activity','rating','time_spent','time_of_day','weekend']]
final_data_kept

,User_id,Customer Age,activity,rating,time_spent,time_of_day,weekend
0,1,53,city_tour,1,100,morning,False
1,2,74,spa,1,169,morning,True
2,3,60,yoga,1,37,morning,False
3,4,41,spa,5,36,evening,False
4,5,66,business_center,4,86,afternoon,False
...,...,...,...,...,...,...,...
9995,9996,71,main_restaurant,2,162,evening,False
9996,9997,33,main_restaurant,3,68,morning,False
9997,9998,29,room_service,4,111,morning,True
9998,9999,78,tennis_court,3,138,afternoon,False


In [94]:
text_columns = ['activity', 'time_of_day']
for col in text_columns:
    unique_values = final_data[col].unique().tolist()
    # Train Word2Vec on unique values of each column
    model = Word2Vec(sentences=[[word] for word in unique_values], vector_size=1, min_count=1, sg=0)
    # Map vectors back to the column
    final_data[col] = final_data[col].map(lambda x: model.wv[x][0])

# Prepare features for cosine similarity
numeric_features = ['Customer Age', 'rating', 'time_spent', 'weekend']
encoded_features = [col for col in text_columns]
all_features = numeric_features + encoded_features

# Compute cosine similarity for all users based on features
features = final_data[all_features]
cosine_sim_matrix = cosine_similarity(features)

# Replace feature values with cosine similarity values
cosine_sim_df = pd.DataFrame(cosine_sim_matrix, index=final_data['User_id'], columns=final_data['User_id'])

# Create a DataFrame where cosine similarity replaces feature values
cosine_sim_features = features.copy()
for i, col in enumerate(all_features):
    # Replace column values with cosine similarity values for each user
    cosine_sim_features[col] = cosine_sim_matrix[i, :]

# Retain original column structure with cosine similarity values
cosine_sim_features['User_id'] = final_data['User_id']


In [95]:
cosine_sim_features

,Customer Age,rating,time_spent,weekend,activity,time_of_day,User_id
0,1.000000,0.997163,0.862383,0.931612,0.985653,0.996401,1
1,0.997163,1.000000,0.822143,0.902463,0.970390,0.999888,2
2,0.862383,0.822143,1.000000,0.982818,0.934362,0.817568,3
3,0.931612,0.902463,0.982818,1.000000,0.979437,0.899747,4
4,0.985653,0.970390,0.934362,0.979437,1.000000,0.968590,5
...,...,...,...,...,...,...,...
9995,0.997228,0.999944,0.822417,0.903039,0.970671,0.999800,9996
9996,0.998888,0.998576,0.843667,0.920540,0.979502,0.998308,9997
9997,0.972888,0.987235,0.722948,0.827441,0.921494,0.988573,9998
9998,0.999542,0.994720,0.875728,0.941735,0.990036,0.993883,9999


In [96]:
def build_user_profiles(data):
    """Build user profiles based on activity patterns"""

    user_profiles = data[['User_id','Customer Age','activity','rating','time_spent','time_of_day','weekend']]

    # One-hot encode categorical features
    user_profiles = pd.get_dummies(user_profiles, columns=['activity', 'time_of_day','weekend'])

    # Assuming 'user_profiles' is your DataFrame with user data
    feature_cols = user_profiles.columns[1:] # Exclude 'User_id'
    features = user_profiles[feature_cols].values

    # Calculate cosine similarity matrix
    similarity_matrix = cosine_similarity(features)

    user_profiles = user_profiles.set_index('User_id')

    return similarity_matrix, user_profiles

In [97]:
def get_similar_users(data,User_id, n=5,similarity_matrix=None):
  """Find similar users based on behavior patterns"""
  if similarity_matrix is None:
    similarity_matrix,user_profiles = build_user_profiles(data)

  user_idx = user_profiles.index.get_loc(User_id)
  user_similarities = similarity_matrix[user_idx]

  similar_user_indices = user_similarities.argsort()[::-1][1:n+1]
  similar_users = user_profiles.index[similar_user_indices]

  return similar_users

In [98]:
def get_recommendations(data, User_id, activity=None, n=5):
  """Generate recommendations for a user"""
  similar_users = get_similar_users(data,User_id)

  # Get activities that similar users rated highly
  similar_users_data = data[data['User_id'].isin(similar_users)]

  if activity:
            similar_users_data = similar_users_data[
                similar_users_data['activity'] == activity
            ]

  # Calculate average ratings for activities
  recommendations = similar_users_data.groupby('activity').agg({
            'rating': 'mean',
            'time_spent': 'mean'
        }).sort_values('rating', ascending=False)

  # Filter out activities the user has already tried
  user_activities = set(data[
            data['User_id'] == User_id
        ]['activity'])
  new_activities = recommendations[
            ~recommendations.index.isin(user_activities)
        ]

  return new_activities.head(n)

In [99]:
new_activities = get_recommendations(final_data_kept,1)
new_activities

,rating,time_spent
activity,,
golf,1.0,107.0
gym,1.0,140.0
tennis_court,1.0,150.0
